In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv('../input/zomato-bangalore-restaurants/zomato.csv')

In [ ]:
df.head(3)

In [ ]:
df.drop(['url','phone','dish_liked'], axis=1,inplace=True)

In [ ]:
df.head(3)

In [ ]:
df.columns

In [ ]:
df= df.rename(columns={'rate':'rating','approx_cost(for two people)':'cost','listed_in(city)':'city','listed_in(type)':'type'})

In [ ]:
df.head(3)

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(how='any',inplace=True)

In [ ]:
df.shape


In [ ]:
df.dtypes

In [ ]:
df['cost'].unique()

# Data Cleaning

### Lets look into the cost column

In [ ]:
df['cost']= df['cost'].astype(str)
df['cost']=df['cost'].apply(lambda x: x.replace(',',''))
df['cost']= df['cost'].astype(float)

In [ ]:
df.cost.value_counts()

### Lets go through rating column


In [ ]:
df['rating'].unique()

In [ ]:
## From the above unique values there are values like 'NEW' in third row, '-' in fifth row, and there are space in between rating

In [ ]:
df= df.loc[df['rating']!='NEW']
df=df.loc[df['rating']!='-'].reset_index(drop=True)
remove_slash= lambda x: x.replace('/5','') if type(x)==np.str else x
df['rating'] = df['rating'].apply(remove_slash).str.strip().astype(float)
df['rating']

In [ ]:
df['rating'].dtype

In [ ]:
df.head()

In [ ]:
df['name']= df['name'].apply(lambda x: x.title())
df['online_order'].replace(('Yes','No'),(True,False),inplace=True)
df['book_table'].replace(('Yes','No'),(True,False),inplace=True)

In [ ]:
df.head(3)

In [ ]:
def encode(df):
    for column in df.columns[~df.columns.isin(['rating','cost','votes'])]:
        df[column]= df[column].factorize()[0]
    return df
df_en= encode(df.copy())
df_en.head()

In [ ]:
plt.figure(figsize=[12,8])
sns.heatmap(df_en.corr(), annot=True,cmap='viridis')
plt.show()

# Exploratory Data analysis

In [ ]:
plt.figure(figsize=[8,6])
sns.countplot(df['online_order'], palette='cubehelix')
plt.show()

In [ ]:
plt.figure(figsize=[10,8])
plt.pie(df['online_order'].value_counts(), labels=['Accepted', 'Not Accepted'], autopct='%1.1f%%', colors=['skyblue', 'pink'],
       startangle=90)
plt.title('Online order')
plt.show()

### Table booking

In [ ]:
plt.figure(figsize=[14,8])
sns.countplot(df['book_table'])
plt.title('Table booking')
plt.show()

### Table booking vs Rating

In [ ]:
pd.crosstab(df['rating'], df['book_table']).sum()

In [ ]:
plt.rcParams['figure.figsize'] = (13, 9)
cr= pd.crosstab(df['rating'], df['book_table'])
cr.div(cr.sum(1).astype(float),axis=0).plot(kind='bar', stacked=True)
plt.title('Booking table vs Rating')
plt.show()

### Locations Wise

In [ ]:
plt.figure(figsize=[14,8])
sns.countplot(df['city'])
plt.title('Restaurents at different locations')
plt.xticks(rotation=90)
plt.show()

#### BTM has the highest number of restaurents

In [ ]:
df.columns

In [ ]:
pd.crosstab(df['rating'], df['rest_type']).plot(kind='bar', stacked=True)
plt.legend().remove()

In [ ]:
df['type'].value_counts()

In [ ]:
df['type'].value_counts(normalize=True)*100

In [ ]:
plt.figure(figsize=[12,8])
sns.countplot(df['type'])
plt.title('Restaurent Type')
plt.show()

#### Delivery and Dine-out's are more popular

In [ ]:
pd.crosstab(df['rating'], df['type']).plot(kind='bar',stacked=True)
plt.title('Restaurent type vs rating')
plt.show()

In [ ]:
plt.figure(figsize=[14,10])
sns.countplot(df['location'])
plt.title('No of restaurents in location')
plt.xticks(rotation=90)
plt.show()

In [ ]:
df['rest_type'].unique()

In [ ]:
sns.countplot(df['rest_type'])
plt.title('Restaurent type')
plt.xticks(rotation=90)
plt.show()

#### Quick-Bites and casual dining are the most popular restaurent type

In [ ]:
df['name'].value_counts()[:20]

In [ ]:
plt.figure(figsize=[15,7])
chains=df['name'].value_counts()[:20]
sns.barplot(x=chains,y=chains.index)
plt.title('Top 20 restaurents')
plt.xlabel('No of outlets')
plt.show()

In [ ]:
df.head()

In [ ]:
df.drop(['address','name','reviews_list','menu_item'], axis=1, inplace=True)

In [ ]:
df.head(3)

In [ ]:
df_rating=df.sort_values('rating', ascending=False)
df_rating.head()

In [ ]:
df_votes=df.sort_values('votes', ascending=False)
df_votes.head()

In [ ]:
df_cost=df.sort_values('cost', ascending=False)
df_cost.head()

In [ ]:
sns.barplot(x=df['type'],y=df['cost'],data=df)
plt.title('Restaurent type vs cost')
plt.show()

#### Drinks and nightlife is expensive, next folows Buffet and Pubs and bars 

In [ ]:
sns.barplot(x=df['city'],y=df['cost'],data=df)
plt.xticks(rotation=90)
plt.title('Restaurent type vs cost')
plt.show()

#### The Church street's restaurents are expensive and next follows Brigade road, Lavelle road,MG road, and Residency road

In [ ]:
sns.barplot(x=df['location'],y=df['cost'],data=df)
plt.xticks(rotation=90)
plt.title('Restaurent type vs cost')
plt.show()

#### The restaurents in Sankey road are expensive

#### Lets dive deep into the data.
1) which type of restaurents fall under below rating==2

In [ ]:
df['rating'].describe()

In [ ]:
df_rating_2= df[df['rating']<=2]
df_rating_2

In [ ]:
df_cuisine= df[(df['cuisines']=='Finger Food') & (df['rating']<=2)]
df_cuisine

In [ ]:
df_low_rating= df.sort_values('rating')
df_low_rating

In [ ]:
sns.barplot(df['city'], df['rating'])
plt.xticks(rotation=90);

In [ ]:
df['cuisines'].unique()

In [ ]:
df['cuisines'].value_counts()[:20].plot.pie(autopct='%1.0f%%')
plt.title('Top 20 cuisines')

#### North Indian and chinese are the most preferable food

In [ ]:
pd.crosstab(df['rating'],df['city']).plot(kind='bar', stacked=True)